In [1]:
import csv
from collections import Counter, defaultdict
import itertools
import pandas as pd
import numpy as np

In [2]:
filename = 'bipartite.suites_only.el.csv'
room2year = dict() # room to grad year mapping
room2majors = defaultdict(Counter) # room to counter of majors in the room
all_majors = set() # set of majors
with open(filename, newline='') as f:
    reader = csv.DictReader(f)
    for line in reader:
        room, major, weight, year = line['room'], line['major'], int(line['weight']), line['year']
        if room not in room2year:
            room2year[room] = year
        room2majors[room][major] += weight
        all_majors.add(major)

In [3]:
all_majors = sorted(list(all_majors))
all_majors_map = {m : i for i, m in enumerate(all_majors)} # map major to index

In [4]:
# major2category = {"Undeclared":"Undeclared", "Economics":"Economics", "Political Science":"Political Science", "History":"History", "Computer Science":"Computer Science, Statistics, Math", "Molecular,Cellular,Dev Biology":"Life Sciences, Environmental Sciences", "Global Affairs":"Political Science", "Psychology":"Social Sciences", "English":"English, Philosophy, Art, Humanities", "Ethics,Politics & Economics":"Area, Multidiciplinary", "Statistics and Data Science":"Computer Science, Statistics, Math", "Mechanical Engineering":"Engineering", "Molecular Biophysics & Biochem":"Engineering", "Ecology & Evolutionary Biology":"Life Sciences, Environmental Sciences", "Biomedical Engineering":"Engineering", "Cognitive Science":"Social Sciences", "History Science, Medicine & PH":"History", "Neuroscience":"Life Sciences, Environmental Sciences", "Environmental Studies":"Life Sciences, Environmental Sciences", "Ethnicity, Race & Migration":"Area, Multidiciplinary", "Mathematics":"Computer Science, Statistics, Math", "Physics (Int.)":"Physical Sciences", "American Studies":"Area, Multidiciplinary", "Applied Mathematics":"Computer Science, Statistics, Math", "Chemical Engineering":"Engineering", "Chemistry":"Physical Sciences", "Elec.Engineering/Computer Sci":"Engineering", "Economics & Mathematics":"Economics", "History of Art":"History", "Music":"English, Philosophy, Art, Humanities", "Physics":"Physical Sciences", "Philosophy":"English, Philosophy, Art, Humanities", "Computing and the Arts":"English, Philosophy, Art, Humanities", "Humanities":"English, Philosophy, Art, Humanities", "Computer Science & Mathematics":"Computer Science, Statistics, Math", "Molecular,Cellular,DevBio(Int)":"Life Sciences, Environmental Sciences", "Sociology":"Social Sciences", "Art":"English, Philosophy, Art, Humanities", "Theater Studies":"English, Philosophy, Art, Humanities", "Anthropology":"Social Sciences", "Architecture":"English, Philosophy, Art, Humanities", "Environmental Engineering":"Engineering", "Electrical Engineering":"Engineering", "Film and Media Studies":"English, Philosophy, Art, Humanities", "Chemistry (Int.)":"Physical Sciences", "Linguistics":"Social Sciences", "Classics":"English, Philosophy, Art, Humanities", "Computer Science & Psychology":"Social Sciences", "Engineering Science-Mechanical":"Engineering", "Mathematics (Int.)":"Computer Science, Statistics, Math", "Mathematics & Philosophy":"Computer Science, Statistics, Math", "East Asian Studies":"Area, Multidiciplinary", "Geology & Geophysics":"Physical Sciences", "African American Studies":"Area, Multidiciplinary", "Astrophysics":"Physical Sciences", "Mathematics & Physics":"Physical Sciences", "Religious Studies":"Area, Multidiciplinary", "Applied Physics":"Physical Sciences", "Sociology (Int.)":"Social Sciences", "Women'sGender&SexualityStudies":"Area, Multidiciplinary", "Engineering Science-Chemical":"Engineering", "Modern Middle Eastern Studies":"Area, Multidiciplinary", "Physics & Philosophy":"English, Philosophy, Art, Humanities", "Russian & E European Studies":"Area, Multidiciplinary", "Comparative Literature":"Area, Multidiciplinary", "French":"Area, Multidiciplinary", "Near Eastern Languages & Civs":"Area, Multidiciplinary", "Political Science (Int.)":"Political Science", "Archaeological Studies":"Social Sciences", "Astronomy":"Physical Sciences", "East Asian Languages & Lits":"Area, Multidiciplinary", "Engineering Science-Electrical":"Engineering", "Lit. and Comparative Cultures":"Area, Multidiciplinary", "Music (Int.)":"English, Philosophy, Art, Humanities", "Spanish":"Area, Multidiciplinary", "African Studies":"Area, Multidiciplinary", "American Studies (Int.)":"Area, Multidiciplinary", "Classical Civilization":"English, Philosophy, Art, Humanities", "Geology & Natural Resources":"Life Sciences, Environmental Sciences", "Global Health":"Political Science", "Italian":"Area, Multidiciplinary", "Judaic Studies":"Area, Multidiciplinary", "Latin American Studies":"Area, Multidiciplinary", "Physics & Geosciences":"Life Sciences, Environmental Sciences", "Special Divisional Major":"Area, Multidiciplinary"}
# categories = ["Economics", "Political Science", "History", "English, Philosophy, Art, Humanities", "Computer Science, Statistics, Math", "Life Sciences, Environmental Sciences", "Social Sciences", "Area, Multidiciplinary", "Engineering", "Physical Sciences", "Undeclared"]
from categories2 import major2category, categories
categories

['Economics',
 'Political Science',
 'History',
 'English, Philosophy, Art, Humanities',
 'Computer Science, Statistics, Math',
 'Life Sciences, Environmental Sciences',
 'Social Sciences',
 'Area, Multidiciplinary',
 'Engineering',
 'Physical Sciences',
 'Undeclared']

In [5]:
# slow
for year in ['2019', '2020', '2021', '2022']:
    edges = list()
    adjacency = pd.DataFrame(0, index=categories, columns=categories, dtype=np.int8)
    for room, majors_counter in room2majors.items():
        if room2year[room] != year:
            continue
        pairs = itertools.combinations(majors_counter.elements(), 2)
        pairs = [sorted(p) for p in pairs]
        for m1, m2 in pairs:
            c1, c2 = major2category[m1], major2category[m2]
            edges.append( [room, year, c1, c2] )
            adjacency.loc[c1, c2] += 1
            if (c2 != c1):
                adjacency.loc[c2, c1] += 1
                
    adjacency.to_csv('collapsed.adjacency.'+year+'.category2.csv', header=True, index=True)
    with open('collapsed.el.'+year+'.category2.csv', 'w') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['room', 'year', 'Source', 'Target'])
        for e in edges:
            writer.writerow(e)

In [6]:
major2category['English']

'English, Philosophy, Art, Humanities'